In [87]:
# !pip install -q selenium lxml tldextract
!pip install -q fake_useragent

In [18]:
import re, os
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
from collections import Counter, defaultdict
from tqdm.auto import tqdm
from urllib.parse import urlparse
from requests import get, head
from random import sample, choice, seed
from time import sleep
from selenium import webdriver
import json
from xmlr import xmlparse, xmliter, XMLParsingMethods
from urllib.parse import urlparse
from multiprocessing import Pool
from functools import partial 
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import threading
from selenium import webdriver
# dir(webdriver)
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from queue import Queue

PATTERN_DOMAIN = '(?:http.*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*'

import tldextract
import langdetect
import json
from xml.etree import ElementTree
# import lxml

import sys
sys.path.append('..')
import utils

In [19]:
def extract_suffix(url):
    obj = tldextract.extract(url)
    return obj.suffix


In [20]:
# test
def test_extract_suffix(): 
    actual = extract_suffix("google.com")
    print(actual)
    actual = extract_suffix("www.google.co.th")
    print(actual)
    actual = extract_suffix("www.google.com:433")
    print(actual)
    actual = extract_suffix("https://www.google.com:433")
    print(actual)
test_extract_suffix()

com
co.th
com
com


In [21]:
def extract_domain(url):
    # t = urlparse(url)
    ext = tldextract.extract(url)
    return '.'.join(part for part in ext[:3] if part)


In [22]:
# test
def test_extract_domain(): 
    actual = extract_domain("google.com")
    print(actual)
    actual = extract_domain("www.google.com")
    print(actual)
    actual = extract_domain("www.google.com:433")
    print(actual)
    actual = extract_domain("https://www.google.com:433")
    print(actual)
test_extract_domain()

google.com
www.google.com
www.google.com
www.google.com


In [23]:
# dataset = {
#     "en": defaultdict(lambda: []),
#     "de": defaultdict(lambda: [])
# }

In [24]:
# counter = Counter()
# with tqdm(total=204522994) as pbar_items:
#     with tqdm(total=67977) as pbar_domains:

#         for d in xmliter('../data/en-de.bicleaner07.tmx', 'tu'):
# #             print(d)
            
#             counter['row_counter'] += 1
#             en_url = d['tuv'][0]['prop']
#             de_url = d['tuv'][1]['prop']

#             # print("en_urls", en_url)
#             # print("de_urls", de_url)

#             if en_url == "unknown" or de_url == "unknown":
#                 counter['unkown_url_counter'] += 1
#                 continue

#             if type(en_url) == list:
#                 en_url = en_url[0]
#             if type(de_url) == list:
#                 de_url = de_url[0]

#             if type(en_url) == str:
#                 en_domain = extract_domain(en_url)
#                 # if not en_domain in dataset['en'].keys():
#                     # dataset['en'][en_domain] = []
#                 # dataset['en'][en_domain].append(en_url)
#                 # else:
#                 if len(dataset['en'][en_domain] ) == 10:
#                     continue
#                 dataset['en'][en_domain].append(en_url)

#             if type(de_url) == str:
#                 de_domain = extract_domain(de_url)

#                 # if not de_domain in dataset['de'].keys():
#                 #     dataset['de'][de_domain] = []
#                 # dataset['de'][de_domain].append(de_url)
#                 # else:
#                 if len(dataset['de'][de_domain]) == 10:
#                     continue
#                 dataset['de'][de_domain].append(de_url)


#             if len(dataset['de'].keys()) > counter['domain_counter']:
#                 counter['domain_counter'] += 1           
#                 pbar_domains.update(1)
                
#             pbar_items.update(1)
#             break

In [25]:
# # Dump to JSON
# with open('../data/en-de.domains.urls.json', 'w', encoding="utf-8") as f:
    
#     json.dump(dataset, f, ensure_ascii=False)

In [9]:
# list(dataset['de'].keys())[-100:]

In [53]:
# len(set(list(dataset['de'].keys())))

## 2. Explore



In [54]:
# Load dataset
with open('../data/en-de.bicleaner07.json', 'r', encoding="utf-8") as f:
    
    %time dataset = json.load(f)

CPU times: user 16.8 s, sys: 30.4 s, total: 47.2 s
Wall time: 1min


In [17]:
def get_status(url):
    try:
        r = head(url, timeout=5)
        code = r.status_code
    except:
        code = 0  # probably bad domain name
    return code

def get_content(url):
    try:
        r = get(url, headers={ 'User-Agent': UserAgent().random })
#         print(r.text)
#         soup = BeautifulSoup(r.content, 'html.parser')
        r.encoding = r.apparent_encoding
        text = r.text 
    except Exception as e:
        print(e)
        text = ""  # probably bad domain name
    
    return text


def substitute_de_to_th(de_url):
    pattern = re.compile(r'\bde\b')

    n = len(pattern.findall(de_url))
    if n == 1:

        th_url = pattern.sub("th", de_url)
        return th_url
    else:
        return None

In [16]:
# get_content('http://japanrailpass.net/th/about_jrp.html#bm_001')
get_content('https://2fish.co/en/church/does-my-church-really-matter/')
# get_c

''

### Find unique domains (registered domain) 

[de]

In [56]:
registered_domains = set()

for domain, urls in dataset['de'].items():
    
    registered_domain = tldextract.extract(domain).registered_domain
    registered_domains.add(registered_domain)    
    

for domain, urls in dataset['en'].items():
    
    registered_domain = tldextract.extract(domain).registered_domain
    registered_domains.add(registered_domain)    

In [57]:
len(registered_domains)

62716

In [58]:
seed(1)
sample(registered_domains, 10)

['paul-albrecht.com',
 'virtualuna.de',
 'tinnemans-scheepswerf.nl',
 'hollerporfidi.it',
 'hotelaquamarin.at',
 'la-padre.ro',
 'straightegyptians.com',
 'bariez.com',
 'globalternative.org',
 'folkmusikhuset.se']

### Find domains (full domain name)

In [59]:
full_domains = set()

for domain, urls in dataset['de'].items():
    full_domains.add(domain)    
    
for domain, urls in dataset['de'].items():
    full_domains.add(domain)    
    

In [60]:
len(full_domains)

77376

In [61]:
seed(1)
sample(full_domains, 10)

['www.ser-foundation.net',
 'elanapet.pl',
 'nadireader.org',
 'anfn.de',
 'www.creamatics.nl',
 'www.a-gross.cz',
 'malereiforschung.de',
 'www.ecg-electro.eu',
 'blog.wiesbaden.de',
 'babynaturalgenderselection.com']

### Find unique domains extension statistics

In [62]:
suffix_counter = Counter()

for domain, urls in dataset['de'].items():
    
    suffix = tldextract.extract(domain).suffix
    suffix_counter[suffix] += 1
    
    
# for domain, urls in dataset['en'].items():
    
#     suffix = tldextract.extract(domain).suffix
#     suffix_counter[suffix] += 1
    
    


In [63]:
suffix_counter.most_common(15)

[('com', 26360),
 ('de', 20093),
 ('org', 3584),
 ('net', 3403),
 ('ch', 2941),
 ('it', 2830),
 ('at', 2426),
 ('cz', 1876),
 ('eu', 1864),
 ('biz', 1091),
 ('nl', 981),
 ('info', 908),
 ('pl', 719),
 ('hu', 691),
 ('sk', 499)]

### Find which domains fall in `\bde\b` pattern

In [64]:
SUB_LANG_PATTERNS = [
    (r'(lang=)(de)', r'\1th'),
    (r'(/)(de)(/)', r'\1th\3'),
    (r'(/)(de)$', r'\1th'),
    (r'^(de)(\.)', r'th\2'), ## lang id as the sub domain
    (r'^(german|ger|ge)(\.)', r'th\2'), ## lang id as the sub domain
    (r'(/)(german|ger|ge)(/)', r'\1th\3'),
    (r'(lang=)((german|ger|ge))', r'\1th'),
]

In [65]:
# มีความหลากหลายใน url format จริงๆ สงสัยต้องไปดู code เฉพาะของเขาแล้วมั้ง น่าจะมี handcraft เยอะอยู่
# ดู en ไปแล้ว ดู mt บ้าง

counter = Counter()

res = []

examples_urls_in_pattern = defaultdict(list)

for domain, urls in tqdm(dataset['de'].items()):
    
    for pattern in SUB_LANG_PATTERNS:
        for url in urls[:2]:
            if re.search(pattern[0], url):
                counter[pattern[0]] += 1

                examples_urls_in_pattern[pattern[0]].append(url)

print('number of URLs that matched with the following patterns:')
counter


number of URLs that matched with the following patterns:


Counter({'(/)(de)(/)': 47792,
         '(/)(german|ger|ge)(/)': 2544,
         '(lang=)(de)': 4736,
         '^(de)(\\.)': 432,
         '(lang=)((german|ger|ge))': 341,
         '(/)(de)$': 206,
         '^(german|ger|ge)(\\.)': 16})

In [66]:
for pattern, urls in examples_urls_in_pattern.items():
    
    print('\npattern:', pattern)
    print('urls (10 samples)', '\n - '.join(list(urls)[:10]))
    print('')


pattern: (/)(de)(/)
urls (10 samples) http://www.buyinsicily.com/de/dienstleistungen-und-kontakte/conoscere-la-sicilia/province/catania/sehenswuerdigkeiten-in-catania.html
 - http://www.buyinsicily.com/de/wer-sind-wir/unserer-geschichte.html
 - http://www.nak.org/de/katechismus/12-gottesdienst-segenshandlungen-und-seelsorge/122-segenshandlungen/1222-konfirmation/12222-konfirmationsgeluebde-und-konfirmationssegen/
 - http://www.nak.org/de/katechismus/5-gottes-gebote/52-gottes-gebote-ausdruck-seiner-liebe/521-die-liebe-zu-gott/
 - www.messaggiamo.com/de/shoes/12732-shoes-you-need-for-summer-holiday.html
 - www.messaggiamo.com/de/gps/8817-spy-matrix-spy-the-ultimate-spy-cell-phone.html
 - knowingallah.com/de/articles/agnostizismus-mit-weniger-zufrieden-sein/
 - http://knowingallah.com/de/articles/warum-gibt-es-so-viele-religionen/
 - www.acidodivertido.com/de/category/en-pareja/
 - www.acidodivertido.com/de/son-dragones-khaleesi-nunca-pueden-ser-domesticados_2/


pattern: (/)(german|ger|

In [31]:
## Send http HEAD Request to that modified URLs ,

### 3) Substitute /de/ as /th/, call a HTTP HEAD request and check returned HTTP[link text](https:// [link text](https://)) staus


In [67]:
# SUB_LANG_PATTERNS = [
#     (r'(lang=)(de)', r'\1th'),
#     (r'(/)(de)(/)', r'\1th\3'),
#     (r'(/)(german|ger|ge)(/)', r'\1th\3'),
#     (r'(lang=)((german|ger|ge))', r'\1th'),
# ]

In [68]:
# def test_substitue_lang():
#     urls = [
#         "balozi.lv/pub/index.php?lang=ger",
#         "balozi.lv/pub/index.php?lang=ge",
#         "balozi.lv/pub/index.php?lang=german",
#         "balozi.lv/de/index.php?lang=de",
#         "balozi.lv/ge/index.php"
#     ]
#     for url in urls:
#         for i in range(0, len(SUB_LANG_PATTERNS)):
#             sub_pattern = SUB_LANG_PATTERNS[i]
#             match, replace = sub_pattern

#             modified_url = re.sub(match, replace, url)

#             print('sub_pattern={},modified_url={}, url={}'.format(i,modified_url, url))
            
#         print('')
        
# test_substitue_lang()

In [132]:
threadLocal = threading.local()


def get_driver(executable_path="chromedriver"):
    driver = getattr(threadLocal, 'driver', None)
    
    if driver is None:
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_argument("--no-sandbox")
        chromeOptions.add_argument('--ignore-certificate-errors')

        driver = webdriver.Chrome(executable_path, options=chromeOptions)
        setattr(threadLocal, 'driver', driver)

    return driver

# driver = get_driver(executable_path="/Users/saiko/chromedriver_78")
# driver

In [137]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


def get_status(url):
    r = None
    url_correct = url
    
    s = requests.Session()
    
    try:
        if 'http://' not in url:
            url_http = 'http://'  + url
            try: 
#                 session = requests.Session()
                r = requests_retry_session(session=s).head(url_http)
                url_correct = url_http
            except:
                        
                try:
    #                 session = requests.Session()                
                    url_https = 'https://'  + url
                    r = requests_retry_session(session=s).head(url_https)
                    url_correct = url_https

                except:
                    pass
        else:
            r = requests_retry_session(session=s).head(url)
        if r == None:
            code= 0
        else:
            code = r.status_code
    except Exception as e:
#         print('exception: get_status()')
#         print('url', url)
#         print(e)
        code = 0  # probably bad domain name
    finally:
        s.close()
    return code, url_correct

def detect_thai_language(text):
    
    pattern = r'[ก-ฮ]'

    if re.search(re.compile(pattern), text):
        return True
    return False


def get_content(url):
    driver = get_driver(executable_path="/Users/saiko/chromedriver_78")
    try:
        driver.get(url)
        text = driver.page_source
    except Exception as e:
        
#         print('exception: get_content()')
#         print('url:', url)
#         print(e)
        return ''
    finally:
        driver.close()
        driver.quit()
    return text


200, modifield_url: http://ajz-engineering.de/th/AJZ-Engineering-GmbH__137/NewsDetail__17520/ , is_thai: True
200, modifield_url: http://ajz-engineering.de/th/AJZ-Engineering-GmbH__137/NewsDetail__17520/ , is_thai: True
200, modifield_url: http://www.museedesblindes.fr/th/la-boutique-du-musee/librairie/item/les-panzers-face-au-debarquement-6-9-juin-1944 , is_thai: True
200, modifield_url: http://www.talayproperty.com/th/the-gallery-condo-jomtien-soi-8-for-rent/l2581 , is_thai: True


In [134]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [135]:

def substitue_lang_worker(match, replace, url):
    
#     print('substitue_lang_worker called.')
#     print('match, replace, url', match, replace, url)
    modified_url = re.sub(match, replace, url)
    status, url_modified_correct = get_status(modified_url)

#     print('status', status)
    is_thai = None
    if int(status) == 200:
        is_thai = detect_thai_language(get_content(url_modified_correct))
        if is_thai:
            print('{}, modifield_url: {} , is_thai: {}'.format(status, url_modified_correct, is_thai))

            
#     result = ()
#     print('result:', result)
#     substitute_lang_worker_callback([result])
    
    result = (is_thai, status, match, url_modified_correct)

#     pattern_counter[match][status] += 1
#     full_domain = utils.extract_full_domain(modified_url)

#     urls_with_status[status][full_domain] = {
#         "is_thai": is_thai,
#         "example_modified_url": [modified_url],
#         'pattern': match,
#     } 
    
#     print('substitue_lang_worker return: ', result)
    return result


In [136]:
urls_with_status = defaultdict(defaultdict)
pattern_counter = defaultdict(Counter)

    
for pattern, urls in examples_urls_in_pattern.items():
      
    for sub_pattern in SUB_LANG_PATTERNS:
        if pattern == sub_pattern[0]:
            match, replace = sub_pattern[0], sub_pattern[1]
            break

    print('')
    print('pattern:', pattern, 'replace:', replace)
    print('number of urls for this pattern:', len(urls))
    print('')

    _substitue_lang_worker = partial(substitue_lang_worker,
                                     match, replace)
#     urls = urls[:1]
#     print(urls)
    results = []
    try:    
        
        with ThreadPoolExecutor(max_workers=8) as executor:
            for url in urls:
#                 print('url:', url)
#                 processes.append(executor.submit(substitue_lang_worker, ))
                results = list(tqdm(executor.map(_substitue_lang_worker, urls), total=len(urls)))

        for result in results:


            is_thai, status, match, modified_url = result

            pattern_counter[match][status] += 1
            full_domain = utils.extract_full_domain(modified_url)

            urls_with_status[status][full_domain] = {
                "is_thai": is_thai,
                "example_modified_url": [modified_url],
                'pattern': match,
            } 
    
    except Exception as e:
        print(e)

print('task done')



pattern: (/)(de)(/) replace: \1th\3
number of urls for this pattern: 47792



200, modifield_url: http://www.oil-painting-online.com/th/expressionism-oil-painting/xu-jiang-oil-painting-art.html , is_thai: True
exception: get_content()
url: http://form.de/th/news/sensing-spaces-architecture-reimagined
HTTPConnectionPool(host='127.0.0.1', port=62745): Max retries exceeded with url: /session/1eefba2c0e65032202ae34fee3bd0a53/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1359a6c50>: Failed to establish a new connection: [Errno 61] Connection refused'))
exception: get_content()
url: http://www.bedzzle.it/th/ferienwohnung/riviera-romagnola
HTTPConnectionPool(host='127.0.0.1', port=62313): Max retries exceeded with url: /session/1d90a4426c65e551ea693e9656df9c25/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1359dccf8>: Failed to establish a new connection: [Errno 61] Connection refused'))
exception: get_content()
url: http://www.bedzzle.it/th/ferienwohnung/provinz-bosnia-erzegovina
HTTPConnectionPool

KeyboardInterrupt: 

exception: get_content()
url: http://www.immobilsarda.com/index.php?s=public&lang=de&p=immobili&filterurl=vendita/-/villa/-/-/-/-/th/immobili/-/-/&pageID=1
HTTPConnectionPool(host='127.0.0.1', port=62836): Max retries exceeded with url: /session/3fdfcd7aa2f119d5da048befe843c279/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x139b91470>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [128]:
pattern_counter

defaultdict(collections.Counter,
            {'(/)(de)(/)': Counter({301: 1}),
             '(/)(german|ger|ge)(/)': Counter({404: 1}),
             '(lang=)(de)': Counter({0: 1}),
             '^(de)(\\.)': Counter({404: 1}),
             '(lang=)((german|ger|ge))': Counter({302: 1}),
             '(/)(de)$': Counter({301: 1}),
             '^(german|ger|ge)(\\.)': Counter({0: 1})})

In [112]:
urls_with_status[200]

defaultdict(None, {})

In [38]:
from datetime import datetime, timezone


def utc_to_local(utc_dt, tz):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

def get_current_datetime(tz):
    
    dt = utc_to_local(datetime.now(), tz=tz)
    return dt

def get_datetime_str(dt):
    
    return dt.strftime('%d.%m.%Y_%H:%M')



In [39]:
filename = 'urls_with_status-{}.json'.format(get_datetime_str(get_current_datetime(tz=7)))
out_path = '../data/' + filename
print('start writing result file to: ', out_path)
with open(out_path, 'w', encoding='utf-8') as f:
    
    json.dump(urls_with_status, f, ensure_ascii=False)
    
print('Done')

start writing result file to:  ../data/urls_with_status-24.11.2019_22:09.json
Done


In [41]:
# cnt = Counter()
# for item, v in http_status_de.items():
# #     if v['status'][0] == 200:
#     cnt[v['status'][0]] += 1

In [42]:
# cnt

### Examples:

- https://2fish.co/th/church/does-my-church-really-matter/

- http://www.oil-painting-online.com/th/expressionism-oil-painting/li-shan-oil-painting-art.html

- https://golfauvietnam.com/orient-golf/?lang=th

- https://gsuite.google.se/intl/th/terms/sla.html

- http://www.xd-cinema.com/th/analytical-comparison-3d-4d-5d-6d-7d-9d-truck-mobile-cinema/

## Detect Language

In [83]:
def fetch_content(url):
    """fetch HTML content (excluded HTML <script> tags)"""
    pass
def detect_language(content):
    """return language id"""
    pass

In [84]:
cnt = Counter()
for item, v in http_status_de.items():
    if v['status'][0] == 200:
        url = v['th_url'][0]
        
        content = fetch_content(url)
        
        